In [4]:
import numpy as np
import pickle
import multiprocessing as mp
import matplotlib.pyplot as plt
%matplotlib inline

from CBAI import MAB, instance_generator, Algo, sigmoid, safe_monotonic_bai

## Safe-Linear Algorithm

- Total number of arms: 10
- Initial points: 0.1
- Boundary values (`M`): 1.5
- Noise parameters (`s_mu` and `s_theta`): $\sim {\cal N}(0,0.5^2)$
- reward parameters (`theta`): [1,0.9,1,...,1]
- safety parameters (`mu`): [1,1.5,5,5...,5]
- safety threshold: 1
- True optimal arm is at index **0**

In [5]:
regret_sum = {}

## We run trials for 20 different seeds
for seed in range(20):
    dim=10
    seed=seed
    np.random.seed(seed)

    init_points = np.array([0.1] + [0.1] *(dim-1)) 
    theta_list = np.array([1, 0.9] + [1] * (dim-2))
    mu_list = np.array([1, 1.5] + [5] * (dim-2))
    M_i_list = np.ones(dim) * 1.5

    bandit = MAB(dim, seed=seed, init_points=init_points, theta_list=theta_list, mu_list=mu_list, M_i_list=M_i_list, s_mu=0.5, s_theta=0.5)
    opt_reward_list = [min(bandit.gamma / bandit.mu_list[i], bandit.M_i_list[i]) * bandit.theta_list[i] for i in range(bandit.d)]

    algo = Algo(bandit, opt_reward_list)
    _ = algo.run_algo()
    res = algo.return_key_stats()
    print(res)
    
    ## You can compute the regret by uncommenting the code block below
#     max_regret = [1-i[0] for i in algo.simple_regret]
#     T = [i[1] for i in algo.simple_regret]
#     regret_sum[seed] = [max_regret,T]

seed 0 dim 10 completed
{'dim': 10, 'gap': [0.0, 0.4, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], 'min_gap': 0.4, 'total arm pulls': 6634.0, 'unsafe pulls': 0, 'true optimal': 0, 'current optimal': 0}
seed 1 dim 10 completed
{'dim': 10, 'gap': [0.0, 0.4, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], 'min_gap': 0.4, 'total arm pulls': 6634.0, 'unsafe pulls': 0, 'true optimal': 0, 'current optimal': 0}
seed 2 dim 10 completed
{'dim': 10, 'gap': [0.0, 0.4, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], 'min_gap': 0.4, 'total arm pulls': 6634.0, 'unsafe pulls': 0, 'true optimal': 0, 'current optimal': 0}
seed 3 dim 10 completed
{'dim': 10, 'gap': [0.0, 0.4, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], 'min_gap': 0.4, 'total arm pulls': 6634.0, 'unsafe pulls': 0, 'true optimal': 0, 'current optimal': 0}
seed 4 dim 10 completed
{'dim': 10, 'gap': [0.0, 0.4, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8], 'min_gap': 0.4, 'total arm pulls': 6634.0, 'unsafe pulls': 0, 'true optimal': 0, 'current optimal': 0}
seed 5 dim

## Safe-Monotonic Algorithm

- Total number of arms: 5
- Seed: 5 (we run 10 trials on same seed to get average, the same cam be done in Safe-Linear but we leave it to the interested researchers to try out)
- Initial points (`a0`): -1.1
- Drug response model: sigmoid / logit function with offset = 0 (`params_f_b0` and `params_g_b0`)
- reward parameters (`params_f_b1`): [1,10,...,10]
- safety parameters (`params_g_b1`): [1,1,...,1]
- safety threshold (`gamma`): 0.3
- safety tolerance (`epssafe`): 0.2
- confidence: 1 - 0.01
- True optimal arm is at index **0**

In [6]:
d = 5
seed = 5
np.random.seed(seed)
delta = 0.01
epssafe = 0.2
gamma = 0.3

a = -1/1*np.log(1/gamma-1)
a0 = [-1.1] * d

best_true = 0

params_f_b1 = np.array([1, 10] + [10] * (d-2))  # reward params
params_f_b0 = np.array([0] * d)
params_g_b1 = np.array([1, 1] + [1] * (d-2)) # cost params
params_g_b0 = np.array([0] * d)
f_func = []
g_func = []

max_val = -1000
best_true = -1
asafe_true = np.zeros(d)
value = np.zeros(d)
for i in range(d):
    a = -1/params_g_b1[i]*(params_g_b0[i] + np.log(1/gamma-1))
    asafe_true[i] = a
    f_func.append(sigmoid)
    g_func.append(sigmoid)
    value[i] = sigmoid(a,params_f_b1[i],params_f_b0[i])
    if sigmoid(a,params_f_b1[i],params_f_b0[i]) > max_val:
        max_val = sigmoid(a,params_f_b1[i],params_f_b0[i])
        best_true = i
        


value[best_true] = -1000
gap = max_val - np.max(value)
print('min_gap: ' + str(gap))

N = 10
correct = 0
total = 0 
total_unsafe = 0
total_epoch = []
total_pulls = []
reward_list = []
regret_list = []
for t in range(N):
    T, best, unsafe, epoch_idx, reward = safe_monotonic_bai(a0,gamma,delta,epssafe,f_func,g_func,params_f_b1,params_f_b0,params_g_b1,params_g_b0,noise_var=0.01)
    if best == best_true:
        correct += 1
    total += T / N
    total_unsafe += unsafe / N
    total_epoch += [epoch_idx]
    total_pulls += [T]
    reward_list += [reward]

print("total arm pulls: {:e}".format(total))
print(f'number of unsafe pulls:{unsafe}')
print(f'correctness: {correct}/{N}')

min_gap: 0.2997910023653128
total arm pulls: 1.227770e+04
number of unsafe pulls:0
correctness: 10/10
